1️⃣ Loads and preprocesses images into tensors ready for VGG16/VGG19.

2️⃣ Computes Gram matrix — captures the "style" of an image (texture, patterns).

3️⃣ Computes Total Variation Loss — makes the generated image smoother.

4️⃣ Provides functions to save and display intermediate or final images.

5️⃣ Loads the correct VGG model and tells which layers to use for content/style features.


In [1]:
import cv2 as cv
import numpy as np
import torch
from torchvision import transforms
import os
import matplotlib.pyplot as plt

In [2]:
from models.definitions.vgg_nets import Vgg16, Vgg19, Vgg16Experimental

In [1]:
IMAGENET_MEAN_255 = [123.675, 116.28, 103.53]
IMAGENET_STD_NEUTRAL = [1, 1, 1]

In [2]:
def load_image(img_path, target_shape=None):
    if not os.path.exists(img_path):
        raise Exception(f'Path does not exist: {img_path}')
    img = cv.imread(img_path)[:, :, ::-1]  # [:, :, ::-1] converts BGR (opencv format...) into RGB

    if target_shape is not None:  # resize section
        if isinstance(target_shape, int) and target_shape != -1:  # scalar -> implicitly setting the height
            current_height, current_width = img.shape[:2]
            new_height = target_shape
            new_width = int(current_width * (new_height / current_height))
            img = cv.resize(img, (new_width, new_height), interpolation=cv.INTER_CUBIC)
        else:  # set both dimensions to target shape
            img = cv.resize(img, (target_shape[1], target_shape[0]), interpolation=cv.INTER_CUBIC)

    # this need to go after resizing - otherwise cv.resize will push values outside of [0,1] range
    img = img.astype(np.float32)  # convert from uint8 to float32
    img /= 255.0  # get to [0, 1] range
    return img
    

Loads an image from disk using OpenCV.

Converts BGR (OpenCV default) → RGB format.

Optionally resizes the image (to given height or shape).

Converts image data to float32 and scales it to range [0, 1].

Returns the processed image as a numpy array.

In [ ]:
def prepare_img(img_path, target_shape, device):
    img = load_image(img_path, target_shape=target_shape)

    # normalize using ImageNet's mean
    # [0, 255] range worked much better for me than [0, 1] range (even though PyTorch models were trained on latter)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.mul(255)),
        transforms.Normalize(mean=IMAGENET_MEAN_255, std=IMAGENET_STD_NEUTRAL)
    ])

    img = transform(img).to(device).unsqueeze(0)

    return img


Uses load_image to load and resize the image.

Converts image into a PyTorch tensor (transforms.ToTensor).

Scales tensor back to [0, 255] and normalizes it using ImageNet mean.

Moves tensor to the specified device (CPU or GPU).

Returns a 4D tensor with batch dimension added (unsqueeze(0))

In [ ]:
def save_image(img, img_path):
    if len(img.shape) == 2:
        img = np.stack((img,) * 3, axis=-1)
    cv.imwrite(img_path, img[:, :, ::-1])  # [:, :, ::-1] converts rgb into bgr (opencv contraint...)

save_image(img, img_path)

Saves a numpy image array to disk as an image file.

If the image is grayscale (2D), it stacks channels to make it RGB.

Converts RGB → BGR format for OpenCV saving.

Calls cv.imwrite to save the image to disk.

Very simple helper — used if you want to save any image.

In [7]:
def generate_out_img_name(config):
    prefix = os.path.basename(config['content_img_name']).split('.')[0] + '_' + os.path.basename(config['style_img_name']).split('.')[0]
    # called from the reconstruction script
    if 'reconstruct_script' in config:
        suffix = f'_o_{config["optimizer"]}_h_{str(config["height"])}_m_{config["model"]}{config["img_format"][1]}'
    else:
        suffix = f'_o_{config["optimizer"]}_i_{config["init_method"]}_h_{str(config["height"])}_m_{config["model"]}_cw_{config["content_weight"]}_sw_{config["style_weight"]}_tv_{config["tv_weight"]}{config["img_format"][1]}'
    return prefix + suffix

generate_out_img_name(config)

Generates an output filename for saving images

Combines parts of content + style image names with hyperparameters.

Adds optimizer name, model name, image height, weights, etc.

If using reconstruction script, uses a simpler filename.

Returns the full filename string — helps organize results clearly.

In [9]:
def save_and_maybe_display(optimizing_img, dump_path, config, img_id, num_of_iterations, should_display=False):
    saving_freq = config['saving_freq']
    out_img = optimizing_img.squeeze(axis=0).to('cpu').detach().numpy()
    out_img = np.moveaxis(out_img, 0, 2)  # swap channel from 1st to 3rd position: ch, _, _ -> _, _, chr

    # for saving_freq == -1 save only the final result (otherwise save with frequency saving_freq and save the last pic)
    if img_id == num_of_iterations-1 or (saving_freq > 0 and img_id % saving_freq == 0):
        img_format = config['img_format']
        out_img_name = str(img_id).zfill(img_format[0]) + img_format[1] if saving_freq != -1 else generate_out_img_name(config)
        dump_img = np.copy(out_img)
        dump_img += np.array(IMAGENET_MEAN_255).reshape((1, 1, 3))
        dump_img = np.clip(dump_img, 0, 255).astype('uint8')
        cv.imwrite(os.path.join(dump_path, out_img_name), dump_img[:, :, ::-1])

    if should_display:
        plt.imshow(dump_img[:,:,::-1]) # convert bgr into rgb for matplotlib
        #plt.imshow(np.uint8(get_uint8_range(out_img)))
        plt.show()

save_and_maybe_display(optimizing_img, dump_path, config, img_id, num_of_iterations, should_display=False)

Saves intermediate or final optimized image to disk.

Converts tensor to numpy array, adds back ImageNet mean, clips values to [0, 255].

Decides when to save (based on saving_freq or final iteration).

If should_display is True, shows image using matplotlib.

Very useful in loop — shows training progress (like epochs).



In [10]:
def prepare_model(model, device):
    # we are not tuning model weights -> we are only tuning optimizing_img's pixels! (that's why requires_grad=False)
    experimental = False
    if model == 'vgg16':
        if experimental:
            # much more flexible for experimenting with different style representations
            model = Vgg16Experimental(requires_grad=False, show_progress=True)
        else:
            model = Vgg16(requires_grad=False, show_progress=True)
    elif model == 'vgg19':
        model = Vgg19(requires_grad=False, show_progress=True)
    else:
        raise ValueError(f'{model} not supported.')

    content_feature_maps_index = model.content_feature_maps_index
    style_feature_maps_indices = model.style_feature_maps_indices
    layer_names = model.layer_names

    content_fms_index_name = (content_feature_maps_index, layer_names[content_feature_maps_index])
    style_fms_indices_names = (style_feature_maps_indices, layer_names)
    return model.to(device).eval(), content_fms_index_name, style_fms_indices_names


\prepare_model(model, device)
Initializes and returns a VGG16 or VGG19 network (pretrained).

Moves the model to device (CPU or GPU).

Does NOT tune model weights — they are frozen.

Returns the model and the layer indices for content + style loss.

Essential step — sets up the model for style transfer.

In [3]:
def gram_matrix(x, should_normalize=True):
    (b, ch, h, w) = x.size()
    features = x.view(b, ch, w * h)
    features_t = features.transpose(1, 2)
    gram = features.bmm(features_t)
    if should_normalize:
        gram /= ch * h * w
    return gram


gram_matrix(x, should_normalize=True)
Computes Gram matrix — captures "style" (texture, patterns).

Takes a feature map tensor → flattens spatial dimensions.

Computes dot-product of features with transpose.

Normalizes the result (if enabled).

Used for calculating style loss in NST (Neural Style Transfer).

In [14]:
def total_variation(y):
    return torch.sum(torch.abs(y[:, :, :, :-1] - y[:, :, :, 1:])) + \
           torch.sum(torch.abs(y[:, :, :-1, :] - y[:, :, 1:, :]))

total_variation(y)

Computes Total Variation Loss — a smoothness regularizer.

Encourages neighboring pixels to have similar colors → less noise.

Adds differences along horizontal and vertical directions.

Returns a scalar — added to total loss during optimization.

Helps avoid too many sharp edges / artifacts in final image.